In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np

In [ ]:
  from google.colab import drive
  drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## The forward and backward passes

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2')


In [ ]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2',
 '/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2',
 '/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2',
 '/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2',
 '/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2']

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [ ]:
#export
from exp.nb_01 import *



def get_data():
    import os
    import torchvision.datasets as datasets
    
    datasets.MNIST.resources = [
        ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
        ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
    ]
    
    root = '../data'
    if not os.path.exists(root):
        os.mkdir(root)
    train_set = datasets.MNIST(root=root, train=True, download=True)
    test_set = datasets.MNIST(root=root, train=False, download=True)
    X_train, X_valid = train_set.data.split([50000, 10000])
    Y_train, Y_valid = train_set.targets.split([50000, 10000])
    
    return (X_train.view(50000, -1) / 256.0), Y_train.float(), (X_valid.view(10000, -1))/ 256.0, Y_valid.float()

#x_train,y_train,x_valid,y_valid = get_data()


#def get_data():
#    path = datasets.download_data(MNIST_URL, ext='.gz')
#    with gzip.open(path, 'rb') as f:
#        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
#    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In the following, I will add mathematical description for the python statements that define the model and the gradients of the loss function with respect to the parameters (weights) of the neural net studied in this notebook. 

Please review what you studied in linear algebra or college math. This mathematical description of the neural net is an application of your basic knowledge of linear algebra.
Without the basic knowledge of linear algebra, that is, the
language of vectors, matrices, transformations, it is not possible to follow this course successfully. You need to spend time to review or study it. I provided good references in the cybercampus site. 

Note that sometimes, I will use variables different from the original notebook, for clarification and for the match with the mathematical description of the network. 

It is really important to be able to understand the mathematical description of the network for "machine learning from foundations". Much of this mathematical description will be the subject matter of the mid-term exam.

In [ ]:
X_train,Y_train,X_valid,Y_valid = get_data()

In [ ]:
len(X_train)

50000

In [ ]:
Y_valid.shape

torch.Size([10000])

X_train is a set of input vectors to the neural net, represented as a matrix: 

X_train =  \\
$X = \begin{bmatrix}
  x^{(1)}  \\ \vdots \\ x^{(m)} 
   \end{bmatrix}
 =  
\begin{bmatrix} 
 x_{1}^{(1)} & \cdots  & x_{n^{[0]}}^{(1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 x_{1}^{(m)} & \cdots &  x_{n^{[0]}}^{(m)}
  \end{bmatrix}
  $

 Here, the index in the parenthesis superfix 1 in $x^{(1)}$ means that input vector $x^{(1)}$ is from  from data sample 1. Each data sample is represented as a row vector $x^{(s)}$.  The suffix 1 in $x_{1}^{(1)}$ means that $x_{1}^{(1)}$  is the first element of vector $x^{(1)}$. $m$ is the number of samples in the dataset:  $m=50,000$. $n^{[0]}$ is the size of the input vector on the 0th layer (the input layer):  $n^{[0]}=784$. In the notebook, $n^{[0]}$ is written as $n0$. 

 Note that 
 $\begin{bmatrix}
  x^{(1)}  \\ \vdots \\ x^{(m)} 
   \end{bmatrix}
 $ is often written as 

  $\begin{bmatrix}
  x^{(1)} & \cdots & x^{(m)} 
   \end{bmatrix}^{T}$ to save space.

  In summary, note that the input to the neural net for training is represented as a matrix x_train. In this representation, the rows of the matrix x_train refer to samples. Some software and papers represent X so that the columns of x_train refer to samples as follows:

  X_train \\
$X = \begin{bmatrix}
  x^{(1)}   \cdots  x^{(m)} 
   \end{bmatrix}
 = 
\begin{bmatrix} 
 x_{1}^{(1)} & \cdots  & x_{1}^{(m)} \\ 
 \vdots          & \cdots & \vdots \\ 
 x_{n^{[0]}}^{(1)} & \cdots &  x_{n^{[0]}}^{(m)}
  \end{bmatrix}
  $

In [ ]:
X_train[0].shape

torch.Size([784])

 The number of nodes in  the input layer =  the size of the 0th layer =  the size of the input vector, x,  = 784 = 28 * 28 = x_train[0].shape.  In the mathematical description of the network, I will use $n^{[0]}$ to refer to the size of the 0th layer: $n^{[0]}$ = 784.

In [ ]:
train_mean,train_std = X_train.mean(),X_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In this notebook, we construct a fully connected (FC) neural net with two layers, one hidden layer and the output layer. The input layer, considered the 0th layer, is not counted in the number of layers. 

The number of nodes in the hidden layer = the size of the hidden layer = 50. The number of nodes in  the input layer =  the size of the 0th layer =  the size of the input vector, x,  = 784 = 28 * 28. The number of nodes in the output layer = the size of the output layer = 1. In the mathematical description of the network, I will use $n^{[0]}$, $n^{[1]}$, $n^{[2]}$ to refer to the size of the 0th layer, the 1st layer, and the second layer, respectively. Hence we have:

$n^{[0]}$ = 784, = $n^{[1]}$ = 50, $n^{[2]}$ =1.

In the notebook, $n^{[0]}$, $n^{[1]}$, $n^{[2]}$ will be written as n0, n1, n2, respectively. 

In [ ]:
X_train = normalize(X_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
X_valid = normalize(X_valid, train_mean, train_std)

In [ ]:
train_mean,train_std = X_train.mean(),X_train.std()
train_mean,train_std

(tensor(3.8966e-08), tensor(1.))

In [ ]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [ ]:
test_near_zero(X_train.mean())
test_near_zero(1-X_train.std())

m is the number of samples in the dataset. 

In [ ]:
X_train.shape

torch.Size([50000, 784])

In [ ]:
m, n0 = X_train.shape
c = Y_train.max()+1
m,n0,c
Y_train.max()

tensor(9.)

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [ ]:
# num hidden
n1 = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

 
The definition of the first layer:

The weighted sum:

Note that matrices are written in capital letters.

$Z^{[1]}
=  X W^{[1]} + b^{[1]}$

$ z^{[1](s)}
=  x^{(s)} W^{[1]} + b^{[1]}, s=1,m = 50000$



 $ z^{[1](s)}
 = \begin{bmatrix} z_{1}^{[1](1)} & \cdots  & z_{n^{[1]}}^{[1](1)}\end{bmatrix}
= \begin{bmatrix} x^{(s)} w_{\bullet  1}^{[1]} & \cdots &  x^{(s)} w_{\bullet  n^{[1]}}^{[1]} \end{bmatrix} 
+ \begin{bmatrix} b_{1}^{[1]} & \cdots & b_{n^{[1]} }^{[1]}\end{bmatrix} \\
$

 $ z^{[1](s)}
 = \begin{bmatrix} z_{1}^{[1](1)} & \cdots  & z_{n^{[1]}}^{[1](1)}\end{bmatrix}
= \begin{bmatrix} 
x_{j}^{(s)} w_{j1}^{[1]} & \cdots &  x_j^{(s)} w_{j n^{[1]}}^{[1]} 
\end{bmatrix} 
+ \begin{bmatrix} b_{1}^{[1]} & \cdots & b_{n^{[1]} }^{[1]}\end{bmatrix} \\
$

Note that $w_{\bullet  1}^{[1]}$ refers to the column $1$ of matrix $W^{[1]}$.
Note that $x_{j}^{(s)} w_{j1}^{[1]}$ is the Einstein summation for weighted sum: 

$ x_{j}^{(s)} w_{j1}^{[1]} = \sum_{j=1}^{n^{[0]}}x_{j}^{(s)} w_{j1}^{[1]} \\
$  

Note that because the input vector $x^{(s)}$ is a row vector, the output of the neuron on the first layer $z^{[1](s)}$  is also a row vector.


 $\begin{bmatrix} 
 z^{[1](1)}  \\ 
 \vdots     \\ 
 z^{[1](m)}  
 \end{bmatrix}
 $  =
$\begin{bmatrix} 
 z_{1}^{[1](1)} & \cdots  & z_{n^{[1]}}^{[1](1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 z_{1}^{[1](m)} & \cdots &  z_{n^{[1]}}^{[1](m)} 
 \end{bmatrix}
 $ \\

 
 = 
$\begin{bmatrix} 
 x_{1}^{(1)} & \cdots  & x_{n^{[0]}}^{(1)} \\ 
 \vdots       & \cdots & \vdots \\ 
 x_{1}^{(m)} & \cdots &  x_{n^{[0]}}^{(m)} 
 \end{bmatrix}
 $
 $ 
\begin{bmatrix} 
 w_{11}^{[2]} & \cdots  & w_{1n^{[1]}}^{[2]} \\ 
 \vdots          & \cdots & \vdots \\ 
 w_{n^{[0]}1}^{[2]} & \cdots &  w_{n^{[0]}n^{[1]}}^{[2]} \end{bmatrix} 
 $ \\

 +
 $ 
  \begin{bmatrix} 
   b_{1}^{[1]}  & \cdots &  b_{n^{[1]}}^{[1]} 
  \end{bmatrix} 
 $ 

=
$\begin{bmatrix} 
 x^{(1)} \cdot  w_{\bullet 1}^{[2]} & \cdots  & x^{(1)} \cdot w_{\bullet n^{[1]}}^{[2]} \\ 
 \vdots       & \cdots & \vdots \\ 
 x^{(m)} \cdot  w_{\bullet 1}^{[2]}  & \cdots & 
 x^{(m)} \cdot w_{\bullet n^{[1]}}^{[2]} 
 \end{bmatrix}
 $
 \\

 +
 $ 
  \begin{bmatrix} 
   b_{1}^{[1]}  & \cdots &  b_{n^{[1]}}^{[1]}  \\
   \vdots  &  \cdots  & \vdots \\
    b_{1}^{[1]}  & \cdots &  b_{n^{[1]}}^{[1]}
  \end{bmatrix} 
 $ 

Note again that $s$ refers to each data sample vector. $n^{[0]}$, $n^{[1]}$ refers to the number of nodes in the 0th layer (input layer) and the 1st layer. 

Note that the last line of the above formula is obtained by the broadcasting of row vector 
$\begin{bmatrix}b_{1}^{[1]}  & \cdots &  b_{n^{[1]}}^{[1]}\end{bmatrix}$.

The activation:

$ 
 A^{[1]}
 =  g( Z^{[1]}) = g(X W^{[1]} + b^{[1]}) $

  $ 
 \begin{bmatrix} 
 a^{[1](1)} \\ 
 \vdots  \\ 
 a^{[1](m)}  
 \end{bmatrix}
=\begin{bmatrix} 
 a_{1}^{[1](1)} & \cdots  & a_{n^{[1]}}^{[1](1)} \\ 
 \vdots       & \cdots & \vdots \\ 
 a_{1}^{[1](m)} & \cdots &  a_{n^{[1]}}^{[1](m)} 
 \end{bmatrix}
=\begin{bmatrix} 
g(z_{1}^{[1](1)}) & \cdots  & g(z_{n^{[1]}}^{[1](1)}) \\ 
 \vdots       & \cdots & \vdots \\ 
 g(z_{1}^{[1](m)}) & \cdots &  g(z_{n^{[1]}}^{[1](m)})  \end{bmatrix} 
 =\begin{bmatrix} 
g(z^{[1](1)})  \\ 
 \vdots     \\ 
 g(z^{[1](m)})  \end{bmatrix} 
$

In our network, $g = relu$. The activations $A^{[1]}$ of the 1st layer will be used as the input to the second layer.


Here,  the index in the square superfix 2 in $z^{[1](1)}$ means that the variable $z^{[1](1)}$ lives on the 1st  layer of the network. 
 The index in the parenthesis superfix 1 in $z^{[1](1)}$ means that the variable $z^{[1](1)}$ is computed from data sample 1.  The suffix 1 in 
  $z_{1}^{[1](1)}$ means that $z_{1}^{[1](1)}$  is the first element of vector $z^{[1](1)}$.  $n^{[i]}$ is the number of the nodes (units or neurons) on the $i$-th layer. The 0th layer of the network refers to the input layer. 

 

The definition of the 2nd layer.

 

$ Z^{[2]}
=  A^{[1]} W^{[2]} + b^{[2]}$

$ z^{[2](s)}
=  a^{[1](s)} W^{[2]} + b^{[2]}, s=1,m = 50000$


$\begin{bmatrix} 
 z_{1}^{[2](1)} & \cdots  & z_{n^{[2]}}^{[2](1)} \\ 
 \vdots          & \cdots & \vdots \\ 
 z_{1}^{[2](m)} & \cdots &  z_{n^{[2]}}^{[2](m)} 
 \end{bmatrix}
 $ \\

 = 
$\begin{bmatrix} 
 a_{1}^{(1)} & \cdots  & a_{n^{[1]}}^{(1)} \\ 
 \vdots       & \cdots & \vdots \\ 
 a_{1}^{(m)} & \cdots &  a_{n^{[1]}}^{(m)} 
 \end{bmatrix}
 $
 $ 
\begin{bmatrix} 
 w_{11}^{[2]} & \cdots  & w_{1n^{[2]}}^{[2]} \\ 
 \vdots          & \cdots & \vdots \\ 
 w_{n^{[1]}1}^{[2]} & \cdots &  w_{n^{[1]}n^{[2]}}^{[2]} \end{bmatrix} 
 $ \\

 +
 $ 
  \begin{bmatrix} 
   b_{1}^{[2]}  & \cdots &  b_{n^{[2]}}^{[2]} 
  \end{bmatrix} 
 $ 



Note that  the activations $A^{[1]}$ of the 1st layer are used as the input to weight matrix $W^{[2]}$. 
 Here, the index in the square superfix 2 in $z^{[2](1)}$ means that the variable $z^{[2](1)}$ lives on the 2nd layer of the network. 
 The index in the parenthesis superfix 1 in $z^{[2](1)}$ means that the variable $z^{[2](1)}$ is computed from data sample 1.  The suffix 1 in $z_{1}^{[2](1)}$ means that $z_{1}^{[2](1)}$  is the first element of vector $z^{[2](1)}$. 
 
 
 
The size of the second layer (the output layer), $n^{[2]}$ is more than one, in general. But in this neural net, there is only one node on the second layer, hence only one element in the output vector $z^{[2](i)}$ on the 2nd layer.  
So we have:


$\begin{bmatrix} 
 z_{1}^{[2](1)} \\ 
 \vdots      \\ 
 z_{1}^{[2](m)}  
 \end{bmatrix}
 $ \\
 = 
$\begin{bmatrix} 
 a_{1}^{[1](1)} & \cdots  & a_{n^{[1]}}^{[1](1)} \\ 
 \vdots       & \cdots & \vdots \\ 
 a_{1}^{(m)} & \cdots &  a_{n^{[1]}}^{[1](m)} 
 \end{bmatrix}
 $
 $ 
\begin{bmatrix} 
 w_{11}^{[2]}  \\ 
 \vdots    \\ 
 w_{n^{[1]}1}^{[2]} 
  \end{bmatrix} 
 $ \\
 +
 $ 
  \begin{bmatrix} 
   b_{1}^{[2]}   
  \end{bmatrix} 
 $ \\
 


In the above, the description of the 1st layer

$ z^{[1](s)}
=  x^{(s)} W^{[1]} + b^{[1]}$ 

assumes that input vector $x^{(s)}$ is written a row vector.
In this case, the vectors in all the layers are considered as row vectors. 

Other people often write input vector $x^{(s)}$ as a column vector as follows:


  x_train \\
$= \begin{bmatrix}
  x^{(1)}   \cdots  x^{(m)} 
   \end{bmatrix}
 = 
\begin{bmatrix} 
 x_{1}^{(1)} & \cdots  & x_{1}^{(m)} \\ 
 \vdots          & \cdots & \vdots \\ 
 x_{n^{[0]}}^{(1)} & \cdots &  x_{n^{[0]}}^{(m)}
  \end{bmatrix}
  $

  In this case, the description of the 1st layer is written as:

  $ z^{[1](s)}
=  W^{[1]} x^{(s)}  + b^{[1]}$ 

Here the output vector $z^{[1](s)}$  is a column vector. Also, the vectors on the other layers are considered as column vectors. If you want to convert this column vector notation to our row vector notation, transpose the equation as follows:

$ {z^{[1](s)}}^{T}
=  ( W^{[1]} x^{(s)} ) ^{T} + {b^{[1]}}^{T}
= {x^{(s)}}^{T}{W^{[1]}}^{T}   + { b^{[1]}}^{T}
$ 

In the row vector notation, the weight matrix W comes next to the row vector. In the column vector notation, the weight matrix comes before the column vector. The column vector notation is the standard in mathematics and other machine learning API's. 
We use the row vector notattion because the fastai code uses the row vector notation.

To train the neural net means to determine the values of the weights $W^{[1]}, W^{[2]}$ and biases $b^{[1]}, b^{[2]}$, so that the loss function L ($W^{[1]}, W^{[2]}$,  $b^{[1]}, b^{[2]}$) is minimized. To do so, we need to initialize them to random numbers chosen appropriately, as follows:

Note that in the noteboo, $W^{[1]}, W^{[2]}, b^{[1]}, b^{[2]}, A ^{[1]} $ are written as W1, W2, b1, b2, A1.

In [ ]:
# simplified kaiming He  init
W1 = torch.randn(n0,n1)/math.sqrt(n0) # n0 is the size of the input vector x, n1 is the number of nodes in the 1st layer
b1 = torch.zeros(n1)
W2 = torch.randn(n1,1)/math.sqrt(n1) # 1 is the number of nodes in the second layer (the output layer)
b2 = torch.zeros(1)

In [ ]:
test_near_zero(W1.mean())
test_near_zero(W1.std()-1/math.sqrt(m))

AssertionError: ignored

In [ ]:
# This should be ~ (0,1) (mean,std)...
X_valid.mean(),X_valid.std()

(tensor(-0.0059), tensor(0.9924))

Function lin() is used to compute

 $ Z^{[2]}
=  A^{[1]} W^{[2]} + b^{[2]}$ and $ Z^{[1]}
=  X W^{[1]} + b^{[1]}$ 

In [ ]:
def lin(X, W, b): return X@W + b

In [ ]:
Z1 = lin(X_valid, W1, b1)

In [ ]:
X_valid.shape

torch.Size([10000, 784])

In [ ]:
#...so should this, because we used kaiming init, which is designed to do this
Z1.mean(),Z1.std()

(tensor(-0.0940), tensor(0.9507))

In [ ]:
def relu(Z): return Z.clamp_min(0.)

In [ ]:
Z1 = lin(X_valid, W1, b1)
A1 = relu( Z1 )

In [ ]:
#...actually it really should be this!
A1.mean(),A1.std()

(tensor(0.3322), tensor(0.5370))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [ ]:
# kaiming init / he init for relu
W1 = torch.randn(n0,n1)*math.sqrt(2/n0)

In [ ]:
W1.mean(),W1.std()

(tensor(0.0003), tensor(0.0506))

In [ ]:
Z1 = lin(X_valid, W1, b1)
A1 = relu( Z1)
A1.mean(),A1.std()

(tensor(0.5202), tensor(0.7672))

In [ ]:
#export
from torch.nn import init

In [ ]:
W1 = torch.zeros(n0,n1)
init.kaiming_normal_(W1, mode='fan_out')
A1 = relu(lin(X_valid, W1, b1))

In [ ]:

W1.shape

torch.Size([784, 50])

In [ ]:
W1.t().shape

torch.Size([50, 784])

In [ ]:
init.kaiming_normal_??

In [ ]:
W1.mean(),W1.std()

(tensor(-0.0004), tensor(0.0513))

In [ ]:
A1.mean(),A1.std()

(tensor(0.1692), tensor(0.2476))

In [ ]:
W1.shape

torch.Size([784, 50])

In [ ]:
import torch.nn

In [ ]:
X_valid[0].shape

torch.Size([784])

In [ ]:
torch.nn.Linear(n0,n1).weight.shape

torch.Size([50, 784])

In [ ]:
torch.nn.Linear.forward??

In [ ]:
torch.nn.functional.linear??

In [ ]:
torch.nn.Conv2d??

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [ ]:
# what if...?
def relu(Z): return Z.clamp_min(0.) - 0.5

In [ ]:
# kaiming init / he init for relu
W1 = torch.randn(n0,n1)*math.sqrt(2./n0 )
A1 = relu(lin(X_valid, W1, b1))
A1.mean(),A1.std()

(tensor(0.0013), tensor(0.7653))

function model(X) computes the output of the neural net Z2 such that

 $ Z^{[2]}
=  A^{[1]} W^{[2]} + b^{[2]}$

$ Z^{[1]}
=  X W^{[1]} + b^{[1]}$ 

In [ ]:
def model(X):
    Z1 = lin(X, W1, b1)
    #print ("Z1=", Z1)
    A1= relu(Z1)
    #print ("A1 = ", A1)
    Z2= lin(A1, W2, b2)
    #print ("Z2 = ", Z2)
    return Z2

In [ ]:
 Z2=model(X_valid[:2])

In [ ]:
%timeit -n 10 _=model(X_valid[0])

In [ ]:
assert model(X_valid).shape==torch.Size([X_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

The loss function $L$:


 $L = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)}) 
$ \\

 $ z^{[2](s)}
=  a^{[1](s)} W^{[2]} + b^{[2]}$

$ a^{[1](s)} = g( z^{[1](s)} )$

 $ z^{[1](s)}
=  x^{(s)} W^{[1]} + b^{[1]}$


Here, $Y\_train$
$ 
= \begin{bmatrix}
 y^{(1)} \\
  \vdots \\
   y^{(m)}
\end{bmatrix}
$

Each $y{(s)}$ is a row vector in general. But in this neural net, it is a scalar. But the form of the equation does not change nor the Python code for it. 

Now to train the neural net, we need to compute the following gradient matrices of $L$ with respec to parameters $W^{[1]}, b^{[1]}, W^{[2]},  b^{[2]}$:

 $\frac{\partial L}{\partial W^{[2]}} = \left[  \frac{\partial L}{\partial w_{ij}^{[2]}} \right]$ \\
 $\frac{\partial L}{\partial b^{[2]}} = \left[ \frac{\partial L}{\partial b_{i}^{[2]}} \right]$ \\
 $\frac{\partial L}{\partial W^{[1]}} = \left[   \frac{\partial L}{\partial w_{ij}^{[1]}} \right]$ 
 \\
 $\frac{\partial L}{\partial b^{[1]}} = \left[ \frac{\partial L}{\partial b_{i}^{[1]}} \right]$
 

During the training of the network,
the gradients are used to update the values of the parameters as follows:

$
W^{[1]} = W^{[1]} + \eta (-\frac{\partial L}{\partial W^{[1]}} ) \\
b^{[1]} = b^{[1]} + \eta (-\frac{\partial L}{\partial b^{[1]}} ) \\
W^{[2]} = W^{[2]} + \eta (-\frac{\partial L}{\partial W^{[2]}} ) \\
b^{[2]} = b^{[2]} + \eta (-\frac{\partial L}{\partial b^{[2]}} ) \\
$



 
 $L = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)}) \\
$ 

 $ z^{[2](s)}
=  a^{[1](s)} W^{[2]} + b^{[2]}$

$\frac{\partial L}{\partial W^{[2]}}
 = \left[    \frac{\partial L}{ \partial w_{ij}^{[2]} } \right]$

$L$ is a function of $z^{[s](2)}$ which is a function of $w_{ij}^{[2]}$:

$  \frac{\partial L}{ \partial w_{ij}^{[2]} }\\
 = \frac{1}{m}\sum_{s=1}^{m}
 \frac{\partial L} {\partial z^{[2](s)}}
 \frac{\partial z^{[2](s)} }{\partial w_{ij}^{[2]} }
 $

The gradient of $L$ with respect to vector $z^{[2](s)}$,
 $\frac{\partial L}{\partial z^{[2](s)} }$, is a row vector:

 $
   \frac{\partial L}{\partial z^{[2](s)} } \\
   = \begin{bmatrix}
    \frac{\partial L }{ \partial z_{1}^{[2](s)} }
       \cdots 
        \frac{\partial L }{ \partial {z_{n^{[2]}}}^{[2](s)} }
    \end{bmatrix} 
   =   2*(z^{[2](s)} - y^{(s)})  
 $ 

The gradient of vector $z^{[2](s)}$  relative to scalar $w_{ij}^{[2]}$, $\frac{\partial z^{[2](s)} }{\partial w_{ij}^{[2]} }$,  is a column vector.



We have:


$\frac{\partial L}{ \partial w_{ij}^{[2]} }
=\frac{\partial L} {\partial z^{[2](s)}}\frac{\partial z^{[2](s)} }{\partial w_{ij}^{[2]} }
=  \sum_{k=1}^{n^{[2]}} \frac{\partial L}{\partial z_{k}^{[2](s)} } \frac{\partial  z_{k}^{[2](s)}  }{\partial w_{ij}^{[2]} } \\
= 
\sum_{k=1}^{n^{[2]}} \frac{\partial L}{\partial z_{k}^{[2](s)} } \frac{ \partial   \sum_{l=1}^{n^{[1]}} a_{l} ^{[1](s)} w_{lk}^{[2]}  }{\partial w_{ij}^{[2]} }\\
= \sum_{k=1}^{n^{[2]}} \frac{\partial L}{\partial z_{k}^{[2](s)} } \frac{   \sum_{l=1}^{n^{[1]}} a_{l} ^{[1](s)} \partial w_{lk}^{[2]}  }{\partial w_{ij}^{[2]} }
=  \frac{\partial L}{\partial z_{j}^{[2](s)} } \frac{    a_{i} ^{[1](s)} \partial w_{ij}^{[2]}  }{\partial w_{ij}^{[2]} }
=  \frac{\partial L}{\partial z_{j}^{[2](s)} } a_{i}^{[1](s)} ( scalar)
$ 

Finally we have

$\frac{\partial L}{\partial W^{[2]}}
 = \left[    \frac{\partial L}{ \partial w_{ij}^{[2]} } \right] 
 =\left[ \frac{1}{m}\sum_{s=1}^{m}  
  \frac{\partial L}{\partial z_{j}^{[2](s)} } a_{i}^{[1](s)} 
   \right] \\
   =\left[ \frac{1}{m}\sum_{s=1}^{m} a_{i}^{[1](s)}  
  \frac{\partial L}{\partial z_{j}^{[2](s)} } 
   \right] \\
   = \frac{1}{m}\sum_{s=1}^{m} {a^{[1](s)}}^{T}  
  \frac{\partial L}{\partial z^{[2](s)} } 
  $


$\frac{\partial L}{\partial b^{[2]}}
  = \frac{\partial L}{\partial Z^{[2]}} \frac{\partial Z^{[2]}}{\partial b^{[2]}} \\
  =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L^{(s)}} {\partial z^{[2](s)}}
   \frac{\partial z^{[2](s)}}{\partial b^{[2]}}
   =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L^{(s)}} {\partial z^{[2](s)}}
   $
 
 



 $L = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)}) 
$ 

$\frac{\partial L}{\partial W^{[1]}}
 = \left[    \frac{\partial L}{ \partial w_{ij}^{[1]} } \right]$

$  \frac{\partial L}{ \partial w_{ij}^{[1]} }
 = \frac{1}{m}\sum_{s=1}^{m}
 \frac{\partial L} {\partial z^{[1](s)}}
 \frac{\partial z^{[1](s)} }{\partial w_{ij}^{[1]} } \\
 $



$\frac{\partial L} {\partial z^{[1](s)}}
 = \frac{\partial L}{\partial a^{[1](s)}}
 \frac{\partial a^{[1](s)}} {\partial z^{[1](s)}}
 = \frac{\partial L} {\partial z^{[2](s)}}
 \frac{\partial z^{[2](s)}} {\partial a^{[1](s)}}
 \frac{\partial a^{[1](s)}} {\partial z^{[1](s)}}
$


$
   \frac{\partial L}{\partial z^{[2](s)} } =2*(z^{[2](s)} - y^{(s)})
 $ 

 $ z^{[2](s)}
=  a^{[1](s)} W^{[2]} + b^{[2]}$

$ a^{[1](s)} = g( z^{[1](s)} )$

 $ z^{[1](s)}
=  x^{(s)} W^{[1]} + b^{[1]}$

$
 \frac{\partial z^{[2](s)}} {\partial a^{[1](s)}}
 =  {W^{[2]}}^{T}
 $

$ \frac{\partial a^{[1](s)}}{\partial z^{[1](s)}}
 =\frac{\partial relu(z^{[1](s)} ) } {\partial z^{[1](s)}} 
 = diag( sgn ( relu'(z^{[1](s)} ) ) )
 $

Here, 

$ sgn( x) = 1$  if $x > 0$, $0$ otherwise. 

So we have

$\frac{\partial L} {\partial z^{[1](s)}}
 =\frac{\partial L}{\partial a^{[1](s)}}
 \frac{\partial a^{[1](s)}} {\partial z^{[1](s)}}
 = 
 (\frac{\partial L} {\partial z^{[2](s)}}
 \frac{\partial z^{[2](s)}} {\partial a^{[1](s)}} )
 \frac{\partial a^{[1](s)}} {\partial z^{[1](s)}} \\
 = ( 2*(z^{[2](s)} - y^{(s)})  {W^{[2]}}^{T} ) \odot 
 sgn(  relu'( z^{[1](s)} ) )  
$

Now

$ \frac{\partial L}{ \partial w_{ij}^{[1]} }$
 =
$\frac{\partial L} {\partial z^{[1](s)}}\frac{\partial z^{[1](s)} }{\partial w_{ij}^{[1]} }
=  \sum_{k=1}^{n^{[1]}} \frac{\partial L}{\partial z_{k}^{[1](s)} } \frac{\partial  z_{k}^{[1](s)}  }{\partial w_{ij}^{[1]} } \\
= 
\sum_{k=1}^{n^{[1]}} \frac{\partial L^{(s)}}{\partial z_{k}^{[1](s)} } \frac{ \partial   \sum_{l=1}^{n^{[1]}} x_{l} ^{(s)} w_{lk}^{[1]}  }{\partial w_{ij}^{[1]} }
= \sum_{k=1}^{n^{[1]}} \frac{\partial L}{\partial z_{k}^{[1](s)} } \frac{   \sum_{l=1}^{n^{[1]}} x_{l} ^{(s)} \partial w_{lk}^{[1]}  }{\partial w_{ij}^{[1]} } \\
=  \frac{\partial L}{\partial z_{j}^{[1](s)} } \frac{    x_{i} ^{(s)} \partial w_{ij}^{[1]}  }{\partial w_{ij}^{[1]} }
=  \frac{\partial L}{\partial z_{j}^{[1](s)} } x_{i}^{(s)}
$

Finally

$\frac{\partial L}{\partial W^{[1]}}
 = \left[    \frac{\partial L}{ \partial w_{ij}^{[1]} } \right] 
  = \left[ \frac{1}{m}\sum_{s=1}^{m}   
  \frac{\partial L}{\partial z_{j}^{[1](s)} } x_{i}^{(s)}
 \right] \\
  = \left[ \frac{1}{m}\sum_{s=1}^{m}   
 x_{i}^{(s)} \frac{\partial L}{\partial z_{j}^{[1](s)} }
 \right] \\
= \frac{1}{m}\sum_{s=1}^{m}  x^{(s)^{T}} \frac{\partial L}{\partial z^{[1](s)} }
$


$\frac{\partial L}{\partial b^{[1]}}
  = \frac{1}{m}\frac{\partial L}{\partial b^{[1]}}  
  =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L} {\partial z^{[1](s)}}
   \frac{\partial z^{[1](s)}}{\partial b^{[1]}} \\
  =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L} {\partial z^{[2](s)}}
    \frac{\partial z^{[2](s)}} {\partial a^{[1](s)}}    
    \frac{\partial a^{[1](s)}} {\partial z^{[1](s)}} I \\
    =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L} {\partial z^{[2](s)}}
    {W^{[2]}}^{T}  
    diag( \frac{\partial relu} {\partial z^{[1](s)}} )\\
     =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L} {\partial z^{[2](s)}}
    {W^{[2]}}^{T} \odot  \frac{\partial relu} {\partial z^{[1](s)}} 
   $
     

 mse() function computes and returns the value of the loss function L:
 
 $L (Z2, Y)  = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)}) \\
$ 


In [ ]:
#export
def mse(Z2, Y_train): return (Z2.squeeze(-1) - Y_train).pow(2).mean()

In [ ]:
Y_train,Y_valid = Y_train.float(),Y_valid.float()

In [ ]:
Z2 = model(X_train)

In [ ]:
loss = mse(Z2, Y_train)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

$L = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)})$

 Compute the gradient of $L$ with respect to $Z^{[2]}$:

$ \frac{ \partial L }{ \partial Z^{[2]} } 
= \begin{bmatrix}
 \frac{ \partial L }{ \partial z^{[2](1)} } \\
 \vdots \\
 \frac{ \partial L }{ \partial z^{[2](m)} }
\end{bmatrix}
$

 $
   \frac{\partial L}{\partial z^{[2](s)} } =2*(z^{[2](s)} - y^{(s)}) / m
 $ 

In [ ]:
def mse_grad(Z2, Y_train): 
    # compute the grad of loss L with respect to Z2 and store it in Z2.g
   

    #print (inp.g.shape)
    # tensor x_train is given a new field that contains its gradient relative y_hat
    m = Z2.shape[0] # the number of rows in Z2
    Z2.g = 2. * (Z2.squeeze() - Y_train).unsqueeze(-1) / m
    

$ \frac{\partial a^{[1](s)}}{\partial z^{[1](s)}}
 =\frac{\partial relu(z^{[1](s)} ) } {\partial z^{[1](s)}} 
 = diag( sgn ( relu'(z^{[1](s)} ) ) )
 $

Here, 

$ sgn( x) = 1$  if $x > 0$, $0$ otherwise.

Represent the diagonal matrix 
$diag( sgn ( relu'(z^{[1](s)} ) ) )$ as a vector $sgn ( relu'(z^{[1](s)} ) )$.




In [ ]:
def relu_grad(Z1, A1):
    # grad of the activation, relu,  with respect to Z1, and store it in Z1.g
    Z1.g = (Z1>0).float() * A1.g


 $L = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)}) 
$ \\

 $ z^{[2](s)}
=  a^{[1](s)} W^{[2]} + b^{[2]}$

$ a^{[1](s)} = g( z^{[1](s)} )$

 $ z^{[1](s)}
=  x^{(s)} W^{[1]} + b^{[1]}$

Function lin_grad2(A1, Z2, W2, b2) computes three gradients at the second layer as follows:

(1) $
\frac{\partial L}{\partial a^{[1](s)}}
=(\frac{\partial L} {\partial z^{[2](s)}}
 \frac{\partial z^{[2](s)}} {\partial a^{[1](s)}} )
= ( 2*(z^{[2](s)} - y^{(s)})  {W^{[2]}}^{T} )$

It will be used to compute the gradient for the layer 1. 

(2)
$\frac{\partial L}{\partial W^{[2]}}
 = \left[    \frac{\partial L^{(s)}}{ \partial w_{ij}^{[2]} } \right] 
= \left[ \frac{1}{m}\sum_{s=1}^{m} a_{i}^{[1](s)} \frac{\partial L^{(s)}}{\partial z_{j}^{[2](s)} } \right] \\
= \frac{1}{m}\sum_{s=1}^{m} \left[  a_{i}^{[1](s)} \frac{\partial L^{(s)}}{\partial z_{j}^{[2](s)} } \right]
= \sum_{s=1}^{m}  a^{[1](s)^{T}} \frac{1}{m}\frac{\partial L^{(s)}}{\partial z^{[2](s)} }$

(3)
$\frac{\partial L}{\partial b^{[2]}}
  = \frac{\partial L}{\partial Z^{[2]}} \frac{\partial Z^{[2]}}{\partial b^{[2]}} \\
  =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L^{(s)}} {\partial z^{[2](s)}}
   \frac{\partial z^{[2](s)}}{\partial b^{[2]}}
   =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L^{(s)}} {\partial z^{[2](s)}}
   $

In [ ]:
def lin_grad2(A1, Z2, W2, b2):
    # grad of L   with respect to A1
    A1.g = Z2.g @ W2.t()
     # grad of L   with respect to W2
    W2.g = (A1.unsqueeze(-1) * Z2.g.unsqueeze(1)).sum(0)
    # grad of L   with respect to b2
    b2.g = Z2.g.sum(0)


 $L = \frac{1}{m}\sum_{s=1}^{m}(z^{[2](s)} - y^{(s)}) 
 \cdot(z^{[2](s)} - y^{(s)}) 
$ \\

 $ z^{[2](s)}
=  a^{[1](s)} W^{[2]} + b^{[2]}$

$ a^{[1](s)} = g( z^{[1](s)} )$

 $ z^{[1](s)}
=  x^{(s)} W^{[1]} + b^{[1]}$

Function lin_grad1(X, Z1, W1, b1) computes three gradients at the 1st layer as follows:

(1) $
\frac{\partial L}{\partial x^{(s)}}
=\frac{\partial L} {\partial z^{[1](s)}}
 \frac{\partial z^{[1](s)}} {\partial x^{(s)}} 
 = ( 2*(z^{[1](s)} - y^{(s)})  {W^{[1]}}^{T} $



(2)
$\frac{\partial L}{\partial W^{[1]}}
 = \left[    \frac{\partial L^{(s)}}{ \partial w_{ij}^{[1]} } \right] 
= \left[ \frac{1}{m}\sum_{s=1}^{m} a_{i}^{[1](s)} \frac{\partial L^{(s)}}{\partial z_{j}^{[1](s)} } \right] \\
= \frac{1}{m}\sum_{s=1}^{m} \left[  a_{i}^{[1](s)} \frac{\partial L^{(s)}}{\partial z_{j}^{[1](s)} } \right]
= \sum_{s=1}^{m}  a^{[1](s)^{T}} \frac{1}{m}\frac{\partial L^{(s)}}{\partial z^{[1](s)} }$

(3)
$\frac{\partial L}{\partial b^{[1]}}
  = \frac{\partial L}{\partial Z^{[1]}} \frac{\partial Z^{[2]}}{\partial b^{[1]}} \\
  =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L^{(s)}} {\partial z^{[1](s)}}
   \frac{\partial z^{[1](s)}}{\partial b^{[1]}}
   =\frac{1}{m}\sum_{s=1}^{m} 
   \frac{\partial L^{(s)}} {\partial z^{[1](s)}}
   $

In [ ]:
def lin_grad1(X, Z1, W1, b1):
    # grad of L with respect to X
    X.g = Z1.g @ W1.t()
     # grad of L   with respect to W1
    W1.g = (X.unsqueeze(-1) * Z1.g.unsqueeze(1)).sum(0)
    # grad of L   with respect to b1
    b1.g = Z1.g.sum(0)

In [ ]:
def forward_and_backward(X_train, Y_train): 
    # forward pass:
    Z1 = X_train @ W1 + b1 # Z1 is a matrix
    A1 = relu(Z1)          # A1 is a matrix
    Z2 = A1 @ W2 + b2      # Z2 is a matrix

 
    # we don't actually need the loss in backward!
    loss = mse(Z2, Y_train) 

    # backward pass:
    # compute the gradient of the loss L with respect to Z2 and store it in Z2.g
    mse_grad(Z2, Y_train) 

    # compute the gradient of Z2 with respect to A1, and store it in A1.g
    lin_grad2(A1, Z2, W2, b2) # xf. Z2 = A1 @ W2 + b2
    relu_grad(Z1, A1)        # cf. A1 = relu(A1)
    lin_grad1(X_train, Z1, W1, b1) #cf. Z1 = X_train @ W1 + b1

In [ ]:
forward_and_backward(X_train, Y_train)

In [ ]:
# Save for testing against later
W1g = W1.g.clone()
W2g = W2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = X_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [ ]:
xt2 = X_train.clone().requires_grad_(True)
W12 = W1.clone().requires_grad_(True)
W22 = W2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(X_train, Y_train):
    # forward pass:
    Z1 = X_train  @ W12 + b12
    A1 = relu(Z1)
    Z2 = A1 @ W22 + b22
    # we don't actually need the loss in backward!
    return mse(Z2, Y_train)

In [ ]:
loss = forward(xt2, Y_train)

In [ ]:
loss.backward()

In [ ]:
test_near(W22.grad, W2g)
test_near(b22.grad, b2g)
test_near(W12.grad, W1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
W1.g,b1.g,W2.g,b2.g = [None]*4
model = Model(W1, b1, W2, b2)

In [ ]:
%time loss = model(X_train, Y_train)

CPU times: user 78.2 ms, sys: 0 ns, total: 78.2 ms
Wall time: 78.4 ms


In [ ]:
%time model.backward()

CPU times: user 2.85 s, sys: 3.19 s, total: 6.04 s
Wall time: 6.06 s


In [ ]:
test_near(W2g, W2.g)
test_near(b2g, b2.g)
test_near(W1g, W1.g)
test_near(b1g, b1.g)
test_near(ig, X_train.g)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
class Model():
    def __init__(self):
        self.layers = [Lin(W1,b1), Relu(), Lin(W2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
W1.g,b1.g,W2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(X_train, Y_train)

CPU times: user 74.2 ms, sys: 4.71 ms, total: 78.9 ms
Wall time: 79 ms


In [ ]:
%time model.backward()

CPU times: user 159 ms, sys: 8.04 ms, total: 168 ms
Wall time: 172 ms


In [ ]:
test_near(W2g, W2.g)
test_near(b2g, b2.g)
test_near(W1g, W1.g)
test_near(b1g, b1.g)
test_near(ig, X_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [ ]:
W1.g,b1.g,W2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(X_train, Y_train)

CPU times: user 80.1 ms, sys: 0 ns, total: 80.1 ms
Wall time: 80.1 ms


In [ ]:
%time model.backward()

CPU times: user 152 ms, sys: 616 µs, total: 152 ms
Wall time: 153 ms


In [ ]:
test_near(W2g, W2.g)
test_near(b2g, b2.g)
test_near(W1g, W1.g)
test_near(b1g, b1.g)
test_near(ig, X_train.g)

### nn.Linear and nn.Module

In [ ]:
#export
from torch import nn

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [ ]:
model = Model(n0, n1, 1)

In [ ]:
%time loss = model(X_train, Y_train)

CPU times: user 82.5 ms, sys: 1.24 ms, total: 83.8 ms
Wall time: 86 ms


In [ ]:
%time loss.backward()

CPU times: user 135 ms, sys: 78.1 ms, total: 213 ms
Wall time: 71.1 ms


## Export

In [ ]:
!pip install fire

     |████████████████████████████████| 92kB 9.2MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=b882ad09b52e4210ba8c88603897014764c94db1e5e1c185a0d7a15fe19e2935
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire


In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2

/content/gdrive/MyDrive/ColabNotebooks/fastaiPart2/course-v3/nbs/dl2


In [ ]:
#!./notebook2script.py 02_fully_connected.ipynb
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
